# Trabajo Integrador: DuckDB

## Instalacion

In [1]:
%pip install duckdb --upgrade


   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.3 MB 2.9 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/11.3 MB 2.2 MB/s eta 0:00:05
   ----- ---------------------------------- 1.6/11.3 MB 2.1 MB/s eta 0:00:05
   ------ --------------------------------- 1.8/11.3 MB 2.1 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/11.3 MB 2.1 MB/s eta 0:00:05
   --------- ------------------------------ 2.6/11.3 MB 1.9 MB/s eta 0:00:05
   ----------- ---------------------------- 3.1/11.3 MB 2.0 MB/s eta 0:00:05
   ------------ --------------------------- 3.7/11.3 MB 2.0 MB/s eta 0:00:04
   --------------- ------------------------ 4.5/11.3 MB 2.3 MB/s eta 0:00:04
   ---------------- ----------------------- 4.7/11.3 MB 2.3 MB/s eta 0:00:03
   ------------------ --------------------- 5.2/11.3 MB 2.1 MB/s eta 0:00:03
   ---------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Creando conexión

Nos permite establecer una conexión a una base de datos, por defecto, si no especificamos su nombre, la base de datos no persistirá y operará en memoria, por lo tanto no se almacenarán las tablas creadas.

In [19]:
import duckdb

BD = duckdb.connect("main.db")

#### Prueba con el dataset

In [20]:
BD.read_csv("dataset/olist_customers_dataset.csv")

BD.sql("SELECT * FROM 'dataset/olist_customers_dataset.csv' LIMIT 10")

┌──────────────────────────────────┬──────────────────────────────────┬──────────────────────────┬───────────────────────┬────────────────┐
│           customer_id            │        customer_unique_id        │ customer_zip_code_prefix │     customer_city     │ customer_state │
│             varchar              │             varchar              │         varchar          │        varchar        │    varchar     │
├──────────────────────────────────┼──────────────────────────────────┼──────────────────────────┼───────────────────────┼────────────────┤
│ 06b8999e2fba1a1fbc88172c00ba8bc7 │ 861eff4711a542e4b93843c6dd7febb0 │ 14409                    │ franca                │ SP             │
│ 18955e83d337fd6b2def6b18a428ac77 │ 290c77bc529b7ac935b93aa66c333dc3 │ 09790                    │ sao bernardo do campo │ SP             │
│ 4e7b3e00288586ebd08712fdd0374a03 │ 060e732b5b29e8181a18229c7b0b2b5e │ 01151                    │ sao paulo             │ SP             │
│ b2b6027bc5c5109e52

#### Conexión con AWS S3

In [21]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
from dotenv import load_dotenv
load_dotenv()  

import os

**NOTA:** Para utilizar AWS S3 es necesario setear las credenciales para acceder al bucket, para eso debemos crear un User en la IAM de AWS, asignarle permisos y finalmente crear las claves de acceso para este usuario, que deben ser almacenadas y es importante que sean de la misma region donde se ubica el bucket

In [ ]:
BD.sql("INSTALL httpfs; LOAD httpfs;")

BD.sql(f"""
SET s3_region='{os.getenv("AWS_REGION")}';
SET s3_access_key_id='{os.getenv("AWS_ACCESS_KEY_ID")}';
SET s3_secret_access_key='{os.getenv("AWS_SECRET_ACCESS_KEY")}';
""")

In [31]:
BD.sql("""
SELECT * FROM read_csv_auto('s3://ti-quadrelli-ribarov/olist_products_dataset.csv') LIMIT 5
""")

┌──────────────────────────────────┬───────────────────────┬─────────────────────┬────────────────────────────┬────────────────────┬──────────────────┬───────────────────┬───────────────────┬──────────────────┐
│            product_id            │ product_category_name │ product_name_lenght │ product_description_lenght │ product_photos_qty │ product_weight_g │ product_length_cm │ product_height_cm │ product_width_cm │
│             varchar              │        varchar        │        int64        │           int64            │       int64        │      int64       │       int64       │       int64       │      int64       │
├──────────────────────────────────┼───────────────────────┼─────────────────────┼────────────────────────────┼────────────────────┼──────────────────┼───────────────────┼───────────────────┼──────────────────┤
│ 1e9e8ef04dbcff4541ed26657ea517e5 │ perfumaria            │                  40 │                        287 │                  1 │              225 │     

### Exploracion del dataset

In [34]:
import pandas as pd

archivos_dataset = os.listdir('dataset')
print("Archivos en el directorio 'dataset':")
for archivo in archivos_dataset:
    print(archivo)

for archivo in archivos_dataset:
    if archivo.endswith('.csv'):
        print(f"\nPrimeras filas de {archivo}:")
        display(pd.read_csv(f'dataset/{archivo}', nrows=5))

Archivos en el directorio 'dataset':
olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_orders_dataset.csv
olist_order_items_dataset.csv
olist_order_payments_dataset.csv
olist_order_reviews_dataset.csv
olist_sellers_dataset.csv
product_category_name_translation.csv

Primeras filas de olist_customers_dataset.csv:


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP



Primeras filas de olist_geolocation_dataset.csv:


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP



Primeras filas de olist_orders_dataset.csv:


...

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53



Primeras filas de olist_sellers_dataset.csv:


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP



Primeras filas de product_category_name_translation.csv:


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [7]:
BD.sql("CREATE TABLE customers AS SELECT * FROM 'dataset/olist_customers_dataset.csv'")